In [1]:
using LinearAlgebra, Distributions, SpecialFunctions, Random, Plots

# Define q-algebra operations for t-exponential family
exp_t(x, t) = t ≈ 1.0 ? exp(x) : max(0.0, (1 + (1-t)*x)^(1/(1-t)))
ln_t(x, t) = t ≈ 1.0 ? log(x) : (x^(1-t) - 1) / (1-t)

function q_product(x, y, t)
    if t ≈ 1.0
        return x * y
    else
        xq = x^(1-t)
        yq = y^(1-t)
        return (xq + yq - 1 > 0) ? (xq + yq - 1)^(1/(1-t)) : 0.0
    end
end

function q_division(x, y, t)
    if t ≈ 1.0
        return x / y
    else
        xq = x^(1-t)
        yq = y^(1-t)
        return (xq - yq + 1 > 0) ? (xq - yq + 1)^(1/(1-t)) : 0.0
    end
end

# Student-t distribution functions
function student_t_pdf(x, μ, σ, ν)
    d = (x - μ) / σ
    return gamma((ν+1)/2) / (gamma(ν/2) * sqrt(π*ν) * σ) * (1 + d^2/ν)^(-(ν+1)/2)
end

function student_t_cdf(x, μ, σ, ν)
    d = TDist(ν)
    return cdf(d, (x - μ)/σ)
end

# Robust likelihood function
function likelihood(f, y, ϵ)
    return ϵ + (1 - 2ϵ) * (sign(y * f) > 0 ? 1.0 : 0.0)
end

# Compute Ψ scaling factor for Student-t distribution
function compute_ψ(σ, ν, t, k=1)
    scale = gamma((ν + k)/2) / ((π * ν)^(k/2) * gamma(ν/2) * abs(σ)^(k/2))
    return scale^(1 - t)
end

# EP algorithm for Student-t process classification
mutable struct StudentTProcessEP
    X::Matrix{Float64}
    y::Vector{Float64}
    v_global::Float64
    kernel::Function
    ϵ::Float64
    τ::Vector{Float64}
    ν::Vector{Float64}
    μ::Vector{Float64}
    Σ::Matrix{Float64}
    K::Matrix{Float64}
end

function StudentTProcessEP(X::Matrix{Float64}, y::Vector{Float64}, v_global::Float64, kernel::Function, ϵ::Float64)
    n = size(X, 1)
    K = kernel_matrix(X, kernel)
    return StudentTProcessEP(X, y, v_global, kernel, ϵ, zeros(n), zeros(n), zeros(n), K, K)
end

function kernel_matrix(X::Matrix, kernel::Function)
    n = size(X, 1)
    K = zeros(n, n)
    for i in 1:n, j in 1:n
        K[i, j] = kernel(X[i, :], X[j, :])
    end
    return K
end

function update_site!(ep::StudentTProcessEP, i::Int)
    n = length(ep.y)
    t = 1 - 2/(ep.v_global + n)
    v_site = ep.v_global + n - 1
    y_i = ep.y[i]
    
    # Step 1: Compute cavity distribution
    μ_i = ep.μ[i]
    σ_i_sq = ep.Σ[i, i]
    σ_i_adj_sq = σ_i_sq * (ep.v_global / v_site)
    ψ_i = compute_ψ(sqrt(σ_i_adj_sq), v_site, t, 1)
    
    τ_minus_i = (v_site)^(-1) * (1/σ_i_adj_sq) * ψ_i - ep.τ[i]
    ν_minus_i = τ_minus_i > 0 ? τ_minus_i * μ_i : 0.0
    
    μ_minus_i = τ_minus_i > 0 ? ν_minus_i / τ_minus_i : μ_i
    σ_minus_i_sq = τ_minus_i > 0 ? v_site / (τ_minus_i * ψ_i) : σ_i_adj_sq
    
    # Step 2: Moment matching for tilted distribution
    z = y_i * μ_minus_i / sqrt(σ_minus_i_sq)
    
    # Compute Z1 and Z2
    d1 = TDist(v_site)
    Z1 = ep.ϵ^t + ((1 - ep.ϵ)^t - ep.ϵ^t) * (1 - cdf(d1, z))
    
    d2 = LocationScale(0, sqrt(v_site/(v_site+2)), d1)
    Z2 = ep.ϵ^t + ((1 - ep.ϵ)^t - ep.ϵ^t) * (1 - cdf(d2, z))
    
    # Compute α and r
    pdf_val = pdf(d1, z)
    α = ((1 - ep.ϵ)^t - ep.ϵ^t) * pdf_val / (Z2 * sqrt(σ_minus_i_sq))
    r = Z1 / Z2
    
    # Update mean and variance
    μ_i_new = μ_minus_i + σ_minus_i_sq * α
    σ_i_new_sq = σ_minus_i_sq * (r - α * μ_i_new)
    
    # Step 3: Site update
    ψ_i_new = compute_ψ(sqrt(σ_i_new_sq), v_site, t, 1)
    τ_i_new = (v_site)^(-1) * (1/σ_i_new_sq) * ψ_i_new - τ_minus_i
    ν_i_new = τ_i_new > 0 ? τ_i_new * μ_i_new : 0.0
    
    # Step 4: Global update
    Δτ = τ_i_new - ep.τ[i]
    Δν = ν_i_new - ep.ν[i]
    
    # Update global natural parameters
    ep.τ[i] = τ_i_new
    ep.ν[i] = ν_i_new
    
    # Update covariance matrix using Woodbury identity
    s = ep.Σ[:, i]
    s_i = s[i]
    denom = 1 + Δτ * s_i
    
    if denom != 0
        ep.Σ -= (Δτ / denom) * s * s'
    end
    
    # Update mean
    ep.μ += Δν * ep.Σ[:, i] - (Δτ * dot(ep.μ, s) / denom) * ep.Σ[:, i]
end

function fit!(ep::StudentTProcessEP, max_iter::Int=10, tol::Float64=1e-3)
    n = length(ep.y)
    prev_μ = copy(ep.μ)
    
    for iter in 1:max_iter
        for i in randperm(n)
            update_site!(ep, i)
        end
        
        # Check convergence
        if norm(ep.μ - prev_μ) < tol
            break
        end
        prev_μ = copy(ep.μ)
    end
end

function predict(ep::StudentTProcessEP, X_test::Matrix{Float64})
    n_train = size(ep.X, 1)
    n_test = size(X_test, 1)
    k_star = [ep.kernel(X_test[i, :], ep.X[j, :]) for i in 1:n_test, j in 1:n_train]
    f_star = k_star * (ep.Σ \ ep.μ)

    println("Predicted latent function values:")
    println(f_star)

    return sign.(f_star)
end

# 各テストポイントがクラス1に属する確率を予測する関数
function predict_proba(ep::StudentTProcessEP, X_test::Matrix{Float64})
    n_train = size(ep.X, 1)
    n_test = size(X_test, 1)
    
    # カーネル行列を計算
    k_star = [ep.kernel(X_test[i, :], ep.X[j, :]) for i in 1:n_test, j in 1:n_train]
    
    # 潜在関数の予測平均値を計算
    f_star = k_star * (ep.Σ \ ep.μ)
    
    # 潜在変数の値を正規分布のCDFに通して確率に変換（プロビット近似）
    # Note: 厳密には予測分散も考慮すべきですが、ここでは平均値のみで近似します
    return cdf.(Normal(), f_star)
end

# Example usage
function rbf_kernel(x1, x2; l=1.0)
    return exp(-sum((x1 - x2).^2) / (2*l^2))
end

# --- データ生成部分 ---
using Random

# 再現性のために乱数シードを固定
Random.seed!(123)

# 各象限あたりに生成する点の数
n_points_per_quadrant = 5
# 点のばらつき具合（ノイズ）
noise = 0.5

# 各象限の中心点と対応するラベル
centers = [[2.0, 2.0], [-2.0, 2.0], [-2.0, -2.0], [2.0, -2.0]]
labels = [1.0, -1.0, -1.0, 1.0]

# 訓練データのからのリストを初期化
X_train_list = []
y_train_list = []

# 各中心点とラベルのペアに対してループ
for (center, label) in zip(centers, labels)
    # 中心の周りに2次元正規分布で点を生成
    dist = MvNormal(center, noise * I(2))
    points = rand(dist, n_points_per_quadrant)
    
    # 生成した点をリストに追加
    push!(X_train_list, points)
    # 対応するラベルをリストに追加
    push!(y_train_list, fill(label, n_points_per_quadrant))
end

# リストを結合して行列(Matrix)とベクトル(Vector)に変換
X = hcat(X_train_list...)'
y = vcat(y_train_list...)

# 新しいテストデータ
X_test = [
    1.0  1.5;  # 第1象限
   -1.2  2.5;  # 第2象限
   -2.5 -1.0;  # 第3象限
    2.2 -2.8   # 第4象限
]
# テストデータの期待される正解ラベル
ep_toy = StudentTProcessEP(X, y, 10.0, (x1,x2)->rbf_kernel(x1,x2; l=0.4), 1e-3)
fit!(ep_toy, 20)

y_test_pred = predict(ep_toy, X_test)
y_test_expected = [1.0, -1.0, -1.0, 1.0]

MethodError: MethodError: no method matching StudentTProcessEP(::Adjoint{Float64, Matrix{Float64}}, ::Vector{Float64}, ::Float64, ::var"#16#17", ::Float64)
The type `StudentTProcessEP` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  StudentTProcessEP(::Any, ::Any, ::Any, ::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any)
   @ Main ~/ws/RR.jl/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sdnNjb2RlLXJlbW90ZQ==.jl:51
  StudentTProcessEP(!Matched::Matrix{Float64}, ::Vector{Float64}, ::Float64, ::Function, ::Float64, !Matched::Vector{Float64}, !Matched::Vector{Float64}, !Matched::Vector{Float64}, !Matched::Matrix{Float64}, !Matched::Matrix{Float64})
   @ Main ~/ws/RR.jl/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sdnNjb2RlLXJlbW90ZQ==.jl:51
  StudentTProcessEP(!Matched::Matrix{Float64}, ::Vector{Float64}, ::Float64, ::Function, ::Float64)
   @ Main ~/ws/RR.jl